Convolutional Neural Network
=============

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save

  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape for convolutional neural network model:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28, 1) (100000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

Let's build a really simple CNN model with one convolutional layer, followed by one fully connected layer (because of the machine's small memory).

In [6]:
batch_size = 128
patch_size = 5
out_channel = 16
hidden_node = 128

initial_learning_rate = 0.4
decay_steps = 1000
decay_rate = 0.65
keep_prob = 0.8
l2_lambda = 0.0001

graph = tf.Graph()

with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  global_step = tf.Variable(0)  # count the number of steps taken
  
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, out_channel], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([out_channel]))

  layer2_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * out_channel, hidden_node], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(0.01, shape=[hidden_node]))

  layer3_weights = tf.Variable(tf.truncated_normal([hidden_node, num_labels], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
  
  def model(data):
    conv = tf.nn.conv2d(data, filter=layer1_weights, strides=[1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    dropped = tf.nn.dropout(x=pool, keep_prob=keep_prob)
    
    shape = dropped.get_shape().as_list()
    reshape = tf.reshape(dropped, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer2_weights) + layer2_biases)
    dropped = tf.nn.dropout(x=hidden, keep_prob=keep_prob)
    
    return tf.matmul(dropped, layer3_weights) + layer3_biases
  
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)) \
    + l2_lambda * (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer3_weights))
    
  learning_rate = tf.train.exponential_decay(learning_rate=initial_learning_rate,
                                             global_step=global_step,
                                             decay_steps=decay_steps,
                                             decay_rate=decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
num_steps = 10001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]

    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Minibatch loss at step 0: 2.418701
Minibatch accuracy: 11.7%
Validation accuracy: 16.1%
Minibatch loss at step 500: 0.666534
Minibatch accuracy: 84.4%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 0.323115
Minibatch accuracy: 93.0%
Validation accuracy: 85.7%
Minibatch loss at step 1500: 0.489066
Minibatch accuracy: 85.9%
Validation accuracy: 86.1%
Minibatch loss at step 2000: 0.523000
Minibatch accuracy: 85.2%
Validation accuracy: 86.7%
Minibatch loss at step 2500: 0.374181
Minibatch accuracy: 90.6%
Validation accuracy: 87.3%
Minibatch loss at step 3000: 0.408997
Minibatch accuracy: 89.1%
Validation accuracy: 87.0%
Minibatch loss at step 3500: 0.422481
Minibatch accuracy: 87.5%
Validation accuracy: 87.4%
Minibatch loss at step 4000: 0.554570
Minibatch accuracy: 83.6%
Validation accuracy: 87.4%
Minibatch loss at step 4500: 0.361762
Minibatch accuracy: 89.8%
Validation accuracy: 87.7%
Minibatc

Below model is too complex (large) to be loaded in current virtual environment. Therefore, please try it from other machine or after expanding the memory of virtual environment. 

In [ ]:
batch_size = 128
patch_size = 5
depth_first = 6
depth_second = 16
hidden_first = 128
hidden_second = 84

initial_learning_rate = 0.4
decay_steps = 1000
decay_rate = 0.65
keep_prob = 0.8

graph = tf.Graph()

with graph.as_default():
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  global_step = tf.Variable(0)
  
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth_first], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth_first]))

  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth_first, depth_second], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(0.01, shape=[depth_second]))

  layer3_weights = tf.Variable(tf.truncated_normal([image_size // 4 * image_size // 4 * depth_second, hidden_first], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(0.01, shape=[hidden_first]))

  layer4_weights = tf.Variable(tf.truncated_normal([hidden_first, hidden_second], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(0.01, shape=[hidden_second]))

  layer5_weights = tf.Variable(tf.truncated_normal([hidden_second, num_labels], stddev=0.1))
  layer5_biases = tf.Variable(tf.constant(0.01, shape=[num_labels]))
  
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    dropped = tf.nn.dropout(x=pool, keep_prob=keep_prob)
    
    conv = tf.nn.conv2d(dropped, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    pool = tf.nn.max_pool(hidden, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    dropped = tf.nn.dropout(x=pool, keep_prob=keep_prob)
    
    shape = dropped.get_shape().as_list()
    reshape = tf.reshape(dropped, [shape[0], shape[1] * shape[2] * shape[3]])
    
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    dropped = tf.nn.dropout(x=hidden, keep_prob=keep_prob)
    
    hidden = tf.nn.relu(tf.matmul(dropped, layer4_weights) + layer4_biases)
    dropped = tf.nn.dropout(x=hidden, keep_prob=keep_prob)
    
    return tf.matmul(dropped, layer5_weights) + layer5_biases
  
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))
    
  learning_rate = tf.train.exponential_decay(learning_rate=initial_learning_rate,
                                             global_step=global_step,
                                             decay_steps=decay_steps,
                                             decay_rate=decay_rate)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [ ]:
# num_steps = 5001

# with tf.Session(graph=graph) as session:
#   tf.initialize_all_variables().run()
#   print("Initialized")
    
#   for step in range(num_steps):
#     offset = (step * batch_size) % (train_labels.shape[0] - batch_size)

#     batch_data = train_dataset[offset:(offset + batch_size), :]
#     batch_labels = train_labels[offset:(offset + batch_size), :]

#     feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
#     _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
#     if (step % 500 == 0):
#       print("Minibatch loss at step %d: %f" % (step, l))
#       print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
#       print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    
#   print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))